In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

### add command path

In [2]:
import sys,numpy,pandas,os
from copy import deepcopy
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')

### import functions

In [28]:
from command.nexus.SPheno import SPheno,block_mapping
from command.scan.scan import scan
from command.multi_thread.MT_SPheno import MT_SPheno
from command.multi_thread.queue_operation import GenerateQueue # generate random queue
from command.pytorch.normalize import normalize
from command.operators.iterable import FlatToList
from command.data_transformer.ArrayToInputQueue import ArrayToInputQueue
from command.data_transformer.InputToPandas import InputToPandas

In [4]:
mold=scan(method='random')
# MinPar
mold.AddScalar('tanB','MINPAR',3,1.,60.)
# Extpar
mold.AddScalar('Lambda','EXTPAR'  ,61  ,1e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('Kappa','EXTPAR'   ,62 ,1.e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR' ,64,-3.e3,3.e3)
mold.AddScalar('mu_eff','EXTPAR'  ,65,100.,1500.)
# Squark mass
mold.AddElement( 'Mq32','MSQ2',(3,3),1e4,4e6)
mold.AddElement( 'Mu32','MSU2',(3,3),1e4,4e6)
mold.AddFollower('Md32','MSD2',(3,3),'Mu32')
# Slepton mass
mold.AddElement( 'Ml32','MSL2',(3,3),1e4,4e6)
mold.AddFollower('Ml22','MSL2',(2,2),'Ml32')
mold.AddFollower('Ml12','MSL2',(1,1),'Ml32')
mold.AddFollower('Me32','MSE2',(3,3),'Ml32')
mold.AddFollower('Me22','MSE2',(2,2),'Ml32')
mold.AddFollower('Me12','MSE2',(1,1),'Ml32')
# Yukawa
mold.AddElement( 'Tt' ,'TU'  ,(3,3),-2000,2000)
mold.AddElement( 'Tb' ,'TD'  ,(3,3),-2000,2000)
mold.AddElement( 'Ttau' ,'TE'  ,(3,3),-2000,2000)
# MSoft
mold.AddScalar('M1','MSOFT',1,50,2000)
mold.AddScalar('M2','MSOFT',2,100,2000)

In [5]:
mold.GetValue('LesHouches.in.NMSSM_low',mapping=block_mapping)
mold.Print()

     Kappa: 0.11
        Tt: 1000.0
      Ml22: 40000.0
  A_Lambda: -1500.0
        M2: 2000.0
        Tb: 1000.0
   A_kappa: -36.0
      Ttau: 0.0
      Me32: 40000.0
      Me12: 40000.0
      Mq32: 1000000.0
      Ml32: 40000.0
      Me22: 40000.0
      Md32: 1000000.0
    Lambda: 0.1
        M1: 500.0
    mu_eff: 965.0
      Mu32: 1000000.0
      tanB: 10.0
      Ml12: 40000.0


In [6]:
order=FlatToList(map(lambda x: list(x.keys()),mold.free_parameter_list.maps))
order

['tanB',
 'Lambda',
 'Kappa',
 'A_Lambda',
 'A_kappa',
 'mu_eff',
 'M1',
 'M2',
 'Mq32',
 'Mu32',
 'Ml32',
 'Tt',
 'Tb',
 'Ttau']

In [119]:
from command.data_transformer.defult_parameter_order import defult_name_order
order=defult_name_order(mold.free_parameter_list)
order

7 1 Tt Kappa
1 7 A_Lambda Tt
1 7 A_Lambda Tt
63 62 A_Lambda Kappa
2 1 M2 A_Lambda
2 7 M2 Tt
8 2 Tb M2
8 7 Tb Tt
1 2 A_kappa M2
64 63 A_kappa A_Lambda
9 2 Ttau M2
9 8 Ttau Tb
3 2 Mq32 M2
3 8 Mq32 Tb
3 7 Mq32 Tt
6 3 Ml32 Mq32
6 8 Ml32 Tb
6 7 Ml32 Tt
1 3 Lambda Mq32
61 64 Lambda A_kappa
1 2 Lambda M2
2 3 M1 Mq32
2 1 M1 A_kappa
1 2 M1 M2
1 2 mu_eff M1
65 64 mu_eff A_kappa
1 2 mu_eff M2
65 61 mu_eff Lambda
4 2 Mu32 M1
4 7 Mu32 Tt
4 6 Mu32 Ml32
4 3 Mu32 Mq32
0 2 tanB M1
0 1 tanB Lambda
0 1 tanB A_Lambda
0 1 tanB Kappa


['tanB',
 'Kappa',
 'A_Lambda',
 'A_kappa',
 'Lambda',
 'mu_eff',
 'M2',
 'M1',
 'Mq32',
 'Mu32',
 'Ml32',
 'Tt',
 'Tb',
 'Ttau']

In [7]:
dimention=len(order)
dimention

14

### normalize & tabulate parameters from mold list to numpy array

In [8]:
Norm=normalize(mold,order=order) # set operation scale=(a,b) to scale parameters to (a,b) range
Norm.scale

(0, 1)

### make a input point queue
*choose one*

In [8]:
# Random points queue by scan's function: Sample
RandSamples=GenerateQueue(mold,lenth=1000) 

In [19]:
# # Random points queue from numpy.random array
# normal_array=numpy.random.rand(1000,dimention) 
# par_array=Norm.D(normal_array)
# RandSamples=ArrayToInputQueue(par_array,mold,order=order)

### multithread SPheno

In [9]:
MTS=MT_SPheno(threads=6,Renew=False)

SPheno file:
->/home/heyangle/Desktop/ScanCraft/ScanCraft/packages/SPheno-4.0.3


In [9]:
MTS.Run(RandSamples,report_interval=1000)#,ReMake='NMSSM_sarah')

Calculations begin at Tue Jun 26 17:05:34 2018
  threads:	6
  points:	1000
Thread-3 got 0 samples
Thread-0 got 0 samples
Thread-2 got 0 samples
Thread-5 got 0 samples
Thread-4 got 0 samples
Thread-1 got 0 samples
  thread-5	runing,        0 points left at Tue Jun 26 17:16:17 2018
All points done. Use 0.183557 hours


KeyError: 'spectrum'

*# read from record*

In [13]:
output_dir=MTS.harvest_dir
os.listdir(output_dir)

['record_180626_171635']

In [81]:
from command.file_operations.GetSamples import GetSamples
samples=GetSamples(path='./output/record_180626_171635/',patterns=['in','out'])
spectr_list=[]
point_list=[]
for sample in samples:
    if len(sample.documents)==2:
        sample.Merge(SPheno.Read(None,sample.documents['out']))
        spectr_list.append(sample)
        inp=deepcopy(mold)
        inp.GetValue(sample.documents['in'],mapping=block_mapping)
        point_list.append(inp)
print(len(spectr_list),len(point_list))

167 167


In [94]:
g_0=InputToPandas(point_list,order=order,title='generation_1')
g_0[:3]

title generation_1                                                             \
block       MINPAR    EXTPAR                                                    
code             3        61        62           63           64           65   
name          tanB    Lambda     Kappa     A_Lambda      A_kappa       mu_eff   
0        27.672458  0.102620  0.131107   734.236108 -1210.995832  1182.715130   
1         2.924756  0.076024  0.169675  1173.376164  -235.757940   858.518302   
2        53.664279  0.111940  0.194700 -1112.492015 -2280.884011  1354.994362   

title                                                                      \
block        MSOFT                       MSQ2          MSU2          MSL2   
code             1            2        (3, 3)        (3, 3)        (3, 3)   
name            M1           M2          Mq32          Mu32          Ml32   
0      1760.206984   706.768164  2.171380e+06  1.906731e+06  1.692713e+06   
1      1464.288135  1821.644835  2.907816e+06  3.784579e+05  3.215624e+06   
2        87.046530   662.821565  1.593503e+06  9.059607e+05  2.974304e+06   

title                                         
block           TU           TD           TE  
code        (3, 3)       (3, 3)       (3, 3)  
name            Tt           Tb         Ttau  
0        95.973483  1600.336209 -1005.868844  
1     -1799.516104  -639.962608   939.381852  
2      1882.305604   827.384057 -1165.733023

In [95]:
g_0=Norm(g_0.values)
print(g_0.shape)

(167, 14)


### Chisqure

In [57]:
def Chisqure(sample):
    X2_h=(sample.MASS[25]-125.01)**2/9
    return X2_h

In [99]:
X2s=numpy.array(list(map(Chisqure,spectr_list))).reshape(-1,1)

In [100]:
# list(zip([s.MASS[25] for s in spectr_list],X2s))

In [102]:
X2s.shape

(167, 1)

### Gene Algorithm

In [113]:
from command.scan.GA import ga
Gene=ga()

In [114]:
storage_0=numpy.hstack([g_0,X2s])
storage_0

array([[4.52075558e-01, 1.01721262e-01, 1.30237003e-01, ...,
        9.00084052e-01, 2.48532789e-01, 1.42405098e+01],
       [3.26229821e-02, 7.50995170e-02, 1.68843451e-01, ...,
        3.40009348e-01, 7.34845463e-01, 2.23888903e+01],
       [8.92614901e-01, 1.11050911e-01, 1.93893965e-01, ...,
        7.06846014e-01, 2.08566744e-01, 7.11741385e-01],
       ...,
       [1.51528892e-01, 9.80710217e-02, 1.68710139e-01, ...,
        7.99404116e-01, 7.64570217e-01, 4.08845030e+00],
       [1.81368265e-01, 5.49474672e-02, 1.11703798e-01, ...,
        1.85166511e-01, 6.43496467e-02, 2.68420601e+00],
       [7.46505075e-01, 3.45552027e-02, 5.05670807e-03, ...,
        5.31274398e-01, 8.55113609e-01, 5.82979142e+02]])

In [115]:
Gene.Generation_new(storage_0[:100])

In [116]:
Gene.Generation_Next()

array([[0.6432 , 0.13585, 0.3182 , ..., 0.92879, 0.23523, 0.90108],
       [0.18621, 0.13248, 0.3968 , ..., 0.02015, 0.64081, 0.92425],
       [0.4096 , 0.12626, 0.05008, ..., 0.52399, 0.90008, 0.24853],
       ...,
       [0.66677, 0.10203, 0.07481, ..., 0.53907, 0.40076, 0.8557 ],
       [0.17944, 0.0157 , 0.02993, ..., 0.21289, 0.86021, 0.29084],
       [0.30881, 0.06501, 0.47655, ..., 0.50208, 0.91133, 0.03189]])